# Binary Neural Network Classification with Multi-layer Perceptrons
## Krishna Thiyagarajan
## ECE - 411 - Computational Graphs for Machine Learning
## Professor Chris Curro
## Homework Assignment #2
## February 6, 2017

In [1]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

warnings.filterwarnings('ignore')

# Hyper Parameters

rateLearn = 1e-3
runs = 200
regConst = 1e-5
displaySteps = 20

noiseSigma = 0.1

totalSamples = 1000
percentTrain = 0.75

numSamp = totalSamples * percentTrain
numTest = totalSamples*(1 - percentTrain)

layers = [2, 20, 40, 20, 2]

In [ ]:

# Data generation function

def data(n):
    t = np.random.uniform(low = 0.0, high = 2* np.pi, size=n)
    c = np.random.randint(0, high=2, size=n)
    x = t * np.cos(t + c * np.pi) + np.random.normal(0, noiseSigma, n)
    y = t * np.sin(t + c * np.pi) + np.random.normal(0, noiseSigma, n)
    return x, y, c

# Function to help declaring variables in model

def defVar(shape, name):
    var = tf.get_variable(name=name,
                          dtype=tf.float32,
                          shape=shape,
                          initializer=tf.random_normal_initializer())
    tf.add_to_collection('modelVars', var)
    tf.add_to_collection('l2', tf.reduce_sum(tf.square(var)))
    return var


# Multi-layer perceptron model with variable layers and neurons

class MultiLayerPercepModel:
    def __init__(self, sess, layers, iterations, learnRate, gamma):
        self.sess = sess
        self.iterations = iterations
        self.learnRate = learnRate
        self.gamma = gamma
        self.layers = layers
        self.buildModel()

    # Build yhat and loss function

    def buildModel(self):
        self.x = tf.placeholder(tf.float32, [None, self.layers[0]])
        self.y = tf.placeholder(tf.float32, [None, self.layers[-1]])

        self.weights = {}
        self.biases = {}

        for ii in range(0, len(self.layers) - 1):
            self.weights[ii] = defVar(name='w%d' % ii, shape=[self.layers[ii], self.layers[ii + 1]])
            self.biases[ii] = defVar(name='b%d' % ii, shape=[self.layers[ii + 1]])

        self.yhat = tf.nn.relu(tf.add(tf.matmul(self.x, self.weights[0]), self.biases[0]))

        for ii in range(1, len(self.layers) - 1):
            self.yhat = tf.nn.relu(tf.add(tf.matmul(self.yhat, self.weights[ii]), self.biases[ii]))

        self.yhat = tf.nn.softmax(self.yhat)

        self.costs = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.yhat, labels=self.y))
        self.l2_penalty = tf.reduce_sum(tf.get_collection('l2'))
        self.loss = self.costs + self.gamma * self.l2_penalty

    def initTrainer(self):
        modelVars = tf.get_collection('modelVars')
        self.optim = (tf.train.AdamOptimizer(learning_rate=self.learnRate).minimize(self.loss, var_list=modelVars))
        self.sess.run(tf.global_variables_initializer())

    def iterateTrainer(self, x, y):
        _, loss = self.sess.run([self.optim, self.loss], feed_dict={self.x: np.transpose(np.asarray(x)), self.y: y})
        return loss

    def train(self, x_samp, y_samp, c_samp):
        for step in range(self.iterations + 1):
            avgCost = 0
            for x, y, c in zip(x_samp, y_samp, c_samp):
                inputs = np.expand_dims(np.asarray([x, y]), axis=1)
                outputs = np.reshape([float(not (c)), float(c)], (1, 2))
                avgCost += self.iterateTrainer(inputs, outputs) / len(x_samp)
            if step % displaySteps == 0:
                print("Step: {:4d}, Loss: {:.9f}".format(step, avgCost))

    # Finds probability that given a point x and y, it's from curve with class 1
    def inferProb(self, x1, y1):
        temp = np.expand_dims(np.asarray([x1, y1]), axis=1)
        return self.sess.run(self.yhat, feed_dict={self.x: np.transpose(temp)})[0][1]

    # Infers class from probability function
    def inferClass(self, x1, y1):
        return self.inferProb(x1, y1) > 0.5


In [ ]:
x_samp, y_samp, c_samp = data(numSamp)
x_test, y_test, c_test = data(numTest)

sess = tf.Session()

model = MultiLayerPercepModel(sess=sess, layers=layers, iterations=runs, learnRate=rateLearn, gamma=regConst)
model.initTrainer()
model.train(x_samp, y_samp, c_samp)

infer = np.array([])

for x, y, _ in zip(x_test, y_test, c_test):
    infer = np.append(infer, model.inferClass(x, y))


# Thanks to Sahil Patel for hinting me towards contours
domain = np.arange(-7, 7, 0.1)
x_cont, y_cont = np.meshgrid(domain, domain)
prob = np.vectorize(model.inferProb)
z_cont = prob(x_cont, y_cont)

accuracy = np.sum(np.equal(c_test,infer))*100/numTest

print("Accuracy: {}%".format(accuracy))

Step:    0, Loss: 0.766119934


In [ ]:
plt.figure()
bound = plt.contour(x_cont, y_cont, z_cont, levels=[0.5])
plt.clabel(bound, inline=1, fontsize=10)

plt.xlabel('x')
plt.ylabel('y', rotation=0)
plt.title('Prediction with Accuracy {}%'.format(accuracy))
plt.plot(x_test[infer == True], y_test[infer == True], 'ro', label='Phased (Class 1)')
plt.plot(x_test[infer == False], y_test[infer == False], 'bo', label = 'Not Phased (Class 0) ')
plt.legend(loc=9, bbox_to_anchor=(0.5,-0.2), ncol=3)
plt.show()